# SasIsaR Model using `Agents`

### Packages

In [1]:
# Data Management 
using DataFrames, DataFramesMeta
using DrWatson

# Statistics
using Random
using Distributions
using StatsBase

# Graphs 
using LightGraphs, SimpleWeightedGraphs
using GraphPlot
 #using Graphs

# Modelling
using Agents
include("./DigitalEpidemiology.jl")

# Data Visualization
using Plots
using AgentsPlots
using PlotThemes

# Python
# ENV["PYTHON"] = "path/to/python"
# Pkg.build("PyCall")
using PyCall
using PyPlot
nx = pyimport("networkx")
np = pyimport("numpy")
hvnx = pyimport("hvplot.networkx")
nw = pyimport("netwulf");

### Utility Functions

In [ ]:
#function rate_to_proportion(r::Float64,t::Float64)
#    1-exp(-r*t)
#end

# Surveillance 
susceptible(x) = count(i == :S for i in x)
infected(x) = count(i == :I for i in x)
recovered(x) = count(i == :R for i in x);

diagnosed_susceptible(x) = count(i == 0 for i in x)
diagnosed_infected(x) = count(i == 1 || i == 2 for i in x)
diagnosed_recovered(x) = count(i == 3 for i in x);

# Diagnostic Capacity 
function constant_diagnostic_capacity(initial_diagnostic_capacity, time_max)
    diagnostic_capacity = [initial_diagnostic_capacity]
    for t in 2:time_max 
        diagnostic_capacity[t] = initial_diagnostic_capacity  
    end
    return diagnostic_capacity
end
    
function linear_diagnostic_capacity(initial_diagnostic_capacity, time_max, slope)
    diagnostic_capacity = [initial_diagnostic_capacity]
    for t in 2:time_max 
        diagnostic_capacity[t] = initial_diagnostic_capacity + slope * t    
    end
    return diagnostic_capacity
end
        
function exponential_diagnostic_capacity(initial_diagnostic_capacity, time_max, a, b)
    diagnostic_capacity = [initial_diagnostic_capacity]
    for t in 2:time_max 
        diagnostic_capacity[t] = initial_diagnostic_capacity + a * exp(b*t)     
    end
    return diagnostic_capacity
end;

### Agent Type 

In [ ]:
# Agent Definition
mutable struct Person <: AbstractAgent
    id::Int64             # ∈ ℕ
    infection::Symbol     # ∈ {S, I, R}
    diagnosis::Int64      # ∈ {0: "not tested" -> S,1: "tested positive" -> I,2:"tested negative once" -> I,3:"tedted negative twice" -> R} 
    perception::Bool      # ∈ 𝔹 = {True, False} = {Symptomatic, Asymptomatic}
end

### Dynamics

In [ ]:
# Symptoms Emergence
function symptoms_emerge!(agent, model)
    # If the susceptible agent doesn't feel any simptoms 
    if agent.infection == :S && agent.perception == false && (rand() ≤ 1 - model.properties[:β_aa] - model.properties[:β_as])
    # Similar symptoms emerge with given onset rate
        agent.perception = true
    # If the infected agent doesn't feel any simptoms 
    #elseif agent.infection == :I && agent.perception == false && (rand() ≤  1 - model.properties[:γ_a])
    # Similar symptoms emerge with given onset rate 
     #   agent.perception = true
    #break
    end
end

# Symptoms Recovery
function symptoms_recover!(agent, model)
    # If the susceptible agent feels simptoms 
    if agent.infection == :S && agent.perception == true && (rand() ≤ 1 - model.properties[:β_ss] - model.properties[:β_sa])
    # Similar symptoms emerge with given onset rate
        agent.perception = false
    # If the infected agent doesn't feel any simptoms 
    #elseif agent.infection == :I && agent.perception == true && (rand() ≤  1 - model.properties[:γ_s])
    # Similar symptoms emerge with given onset rate 
     #   agent.perception = false
    #break
    end
end

# Infection Transmission 
function infection_transmit!(agent, model)
    # If I'm not susceptible, I return
    if agent.infection != :S && return
    end
    # Poisson distribution with rate parameter c
    ncontacts = rand(Poisson(model.properties[:c]))             
    
    if agent.perception == true
            for i in 1:ncontacts
            # Choose random individual
                alter = random_agent(model)
                # If alter is symptomatic infected 
                if alter.infection == :I && alter.perception == true && (rand() ≤ model.properties[:β_ss])
                    # An infection occurs
                    agent.infection = :I
                # If alter is asymptomatic infected 
                elseif alter.infection == :I && alter.perception == false && (rand() ≤ model.properties[:β_sa])
                    agent.infection = :I
                #break 
                end 
            end
    else 
            for i in 1:ncontacts
            # Choose random individual
                alter = random_agent(model)
                # If alter is symptomatic infected 
                if alter.infection == :I && alter.perception == true && (rand() ≤ model.properties[:β_as])
                    # An infection occurs
                    agent.infection = :I
                # If alter is asymptomatic infected 
                elseif alter.infection == :I && alter.perception == false && (rand() ≤ model.properties[:β_aa])
                    agent.infection = :I
                #break 
                end 
            end    
        end
end;

# Infection Recovery 
function infection_recover!(agent, model)
    agent.infection != :I && return
            
    if agent.perception == true && rand() ≤ model.properties[:γ_s]
            agent.infection = :R
            agent.perception = false
    elseif agent.perception == false && rand() ≤ model.properties[:γ_a]
            agent.infection = :R
            agent.perception = false        
    end
end;
         

# Random Diagnostic Strategy 
function diagnose!(model, to_diagnose, diagnostic_capacity)
sample = StatsBase.sample(to_diagnose, diagnostic_capacity; replace=true)
    for agent in sample 
        if agent.diagnosis == 0 
            if agent.infection == :I
                agent.diagnosis = 1
            else
                agent.diagnosis = 0
            end
        end
        if agent.diagnosis == 1 
            if agent.infection == :I
                agent.diagnosis = 1
            else
                agent.diagnosis = 2
            end
        end
        if agent.diagnosis == 2 
            if agent.infection == :I
                agent.diagnosis = 1
            else
                agent.diagnosis = 3
                deleteat!(to_diagnose, agent)
            end
        end
    end
end

    

# Individual Dynamics 
function agent_step!(agent, model)
    symptoms_emerge!(agent, model)
    symptoms_recover!(agent, model)
    infection_transmit!(agent, model)
    infection_recover!(agent, model)
    #diagnose!(model, to_diagnose,diagnostic_capacity)
end;

### Model

In [ ]:
# Model Initialization
function init_model(β_aa::Float64, β_as::Float64, 
                    β_sa::Float64, β_ss::Float64, 
                    γ_a::Float64, γ_s::Float64,
                    c::Float64, 
                    N::Int64, I0::Int64, I0_s::Int64, S0_a::Int64)
    
    I0_a = I0 - I0_s 
    S0_s = N - I0 - S0_a
       
    properties = @dict(β_aa, β_as, 
                       β_sa, β_ss, 
                       γ_a, γ_s, 
                       c)
      
    model = ABM(Person; properties=properties)
    
    for id in 1:N
        if id <= I0_a
            infection = :I
            diagnosis = 0
            perception = false
        elseif id <= I0_a + I0_s
            infection = :I
            diagnosis = 0
            perception = true
        elseif id <= I0 + S0_a
            infection = :S
            diagnosis = 0
            perception = false
        else  #if id <= I0 + S0_a + S0_s
            infection = :S
            diagnosis = 0
            perception = true
        end
        
        person = Person(id, 
                   infection, 
                   diagnosis, 
                   perception)
        person = add_agent!(person, model)
    end
    return model
end;

### Parameters

In [ ]:
# Time 
δt = 0.1
nsteps = 150
tf = nsteps*δt
t = 0:δt:tf;

# Model Parameters
s =  0.3              # Symptomatic damping factor
β_aa = 0.3              # S_a --> I_a transmission rate
β_as = 0.6              # S_a --> I_s transmission rate
β_sa = (1-s) * β_aa     # S_s --> I_a transmission rate
β_ss = (1-s) * β_as     # S_s --> I_s transmission rate
γ_a  = 0.08              # Asymptomatic recovery rate
γ_s  = 0.8             # Symptomatic recovery rate
c = 10.0*δt             # Contact rate 

#γ = rate_to_proportion(0.25,δt);   # Recovery rate

# Initial Conditions
N = 1500
I0 = 20
I0_s = 4
S0_a = N - I0 - 20;

# Set Diagnostic Capacity 
diagnostic_capacity = 15 ;

### Simulation

In [ ]:
# Seed Selection
Random.seed!(1234);

# Model Instantiation 
model = init_model(β_aa, β_as, 
                   β_sa, β_ss, 
                   γ_a, γ_s,
                   c,
                   N, I0, I0_s, S0_a)

# Fill Array of Agents to Test
to_diagnose = DataFrame() # allagents(model)

# Data Collection
to_collect = [(:infection, f) for f in (susceptible, infected, recovered)]
data, _ = run!(model, agent_step!, nsteps; adata = to_collect);

sort!(DataFrame(allagents(model)), :score, rev= true)

### Visualization

In [ ]:
# Data Manipulation
data[!,:t] = t;

# Select Theme
theme(:default)

# Plot
plot(t,data[:,2],label="S",
     xlab="Time",
     ylabel="Populaion",
     title="SasIsaR ABM", 
     legend=:right)
plot!(t,data[:,3],label="I")
plot!(t, data[:,4],label="R")

In [ ]:
to_diagnose = DataFrame(allagents(model))

agents_data_id = to_diagnose.id
agents_data_infection = to_diagnose.infection
agents_data_diagnosis = to_diagnose.diagnosis

In [ ]:
sample_id = StatsBase.sample(to_diagnose.id, diagnostic_capacity; replace=true)

In [ ]:
Array(to_diagnose)

In [ ]:
    for agent.id in sample_id 
        if agent.diagnosis == 0 
            if agent.infection == :I
                agent.diagnosis = 1
            else
                agent.diagnosis = 0
            end
        end
        if agent.diagnosis == 1 
            if agent.infection == :I
                agent.diagnosis = 1
            else
                agent.diagnosis = 2
            end
        end
        if agent.diagnosis == 2 
            if agent.infection == :I
                agent.diagnosis = 1
            else
                agent.diagnosis = 3
                deleteat!(to_diagnose, agent)
            end
        end
    end
end

    



In [ ]:
for i in allagents(model)
    
    println(random_agent(model))
    
end
